In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime as dt 

In [0]:
df = spark.table('default.sample_superstore')
df = df.toPandas()

qnt_linhas = df.shape 
nome_colunas = df.columns
tipo_dados = df.dtypes
estat_resumo = df['Sales'].describe() # valido somente para variaveis quantitativa
qnt_duplicados = df[df.duplicated]
qnt_nulos = df.isnull().sum()

analise_exploratoria = (
    f'Quatidade de linhas por colunas: {qnt_linhas}\n'
    '\n'
    f'Nome das colunas: \n{nome_colunas}\n'
    '\n'
    f'Tipo de dado de cada coluna: \n{tipo_dados}\n'
    '\n'
    f'Resumo estatístico da coluna vendas: \n{estat_resumo}\n'
    '\n'
    f'Dados duplicados: \n{qnt_duplicados}\n'
    '\n'
    f'Dados nulos: \n{qnt_nulos}\n'
)

print(analise_exploratoria)

 **1** - Qual a **cidade** com maior **valor de venda** de produtos da **categoria** Office Supplies?

In [0]:
filtro_categoria = df[df['Category'] == 'Office Supplies']
group_cidade_venda = filtro_categoria.groupby('City')['Sales'].sum()
cidade_maior_venda = group_cidade_venda.sort_values(ascending=False).first_valid_index()
ranking = group_cidade_venda.sort_values(ascending = False)
print(f'Ranking das cidades com maior valor de venda da categoria Office Supplies: \n{ranking}\n'
      f'Sendo a cidade com maior valor {cidade_maior_venda}')

**2** - Qual o total de **vendas** por **data do pedido**? (Em gráfico de barras)

In [0]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
group_vendas_data = df.groupby('Order Date')['Sales'].sum().reset_index()
plt.figure(figsize=(20, 6))
plt.plot(group_vendas_data['Order Date'], group_vendas_data['Sales'], color='green')
plt.title('Total de vendas por data de pedido')
plt.xlabel('Order Date')
plt.ylabel('Sales')
plt.show()